In [1]:
import numpy as np
from stdb import load_db 
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import glob, os

import warnings  
from obspy.geodetics.base import gps2dist_azimuth
from obspy.core.event.catalog import read_events
warnings.filterwarnings("ignore")  
from  PlotCatalog_1 import plot_nonnull_average, plot_null_PCA09, plot_null_PCA_angle_small,plot_null_PCA_angle_large, plot_null_angle
                


In [2]:
PKLpath = '/Volumes/home/Research/STEP/01_Split'
Path = '/Volumes/home/Research/STEP/02_Station_result_csv'
# MAG = 6

SAVEpath = '/Volumes/home/Research/STEP/05_Station_result_statistics'
YearRange= 'SKSvsSKKS'
ver= 4

In [7]:
iiii=0
for STApath in glob.glob(f'{PKLpath}/*.pkl'):
    STApkl = pickle.load(open(STApath, "rb"))
    values = [value for keys, value in STApkl.items() ][0]

    NET = values['network']; STA = values['station']
    if STA == 'CMCY' or STA == 'DGRL' or NET == 'BI' :
        pass
    else:
        newdf = pd.DataFrame()
        for csvpath in glob.glob(f'{Path}/*classify/{NET}*{STA}*result.csv'):
            phase = csvpath.rsplit('_')[4]
            df = pd.read_csv(csvpath)
            df = df[df['Pick']==True][df['Null']==False][df['Quality']!='Poor']
            iiii += len(df)

In [3]:
allSTA = {'Station':[],
         'RCphi':[],'RCdphi':[],
         'RCdt':[],'RCddt':[],
         'SCphi':[],'SCdphi':[],
         'SCdt':[],'SCddt':[],
         'SKSnonNull':[], 'SKSNull':[],
         'SKKSnonNull':[], 'SKKSNull':[],
         'nonNull':[],'Null':[]}


for STApath in glob.glob(f'{PKLpath}/*GNI.pkl'):
    STApkl = pickle.load(open(STApath, "rb"))
    values = [value for keys, value in STApkl.items() ][0]

    NET = values['network']; STA = values['station']
    if STA == 'CMCY' or STA == 'DGRL' :
        pass
    else:
        print(f'{NET}.{STA}')    
        if not os.path.isdir(f'{SAVEpath}/{NET}.{STA}'):
            os.mkdir(f'{SAVEpath}/{NET}.{STA}')
        allSTA['Station'].append(f'{NET}.{STA}')
        stlat = values['latitude']; stlon = values['longitude']

        newdf = pd.DataFrame()
        for csvpath in glob.glob(f'{Path}/*classify/{NET}*{STA}*result.csv'):
            phase = csvpath.rsplit('_')[4]
            df = pd.read_csv(csvpath)
            df = df[df['Pick']==True]
            newdf = newdf.append(df)
            if phase == 'SKS':
                if len(df) !=0:
                    allSTA['SKSnonNull'].append(len( df[df['Null']==False][df['Quality']!='Poor'] ))
                    allSTA['SKSNull'].append(len( df[df['Null']==True] ))
                else:
                    allSTA['SKSnonNull'].append(0)
                    allSTA['SKSNull'].append(0)  
            elif phase == 'SKKS':
                if len(df) !=0:
                    allSTA['SKKSnonNull'].append(len( df[df['Null']==False][df['Quality']!='Poor'] ))
                    allSTA['SKKSNull'].append(len( df[df['Null']==True] ))            
                else:
                    allSTA['SKKSnonNull'].append(0)
                    allSTA['SKKSNull'].append(0)             


        print(len(newdf))
        meanphiRC, stdphiRC, meandtRC, stddtRC, meanphiSC, stdphiSC, meandtSC, stddtSC, noNull, noNonNull = plot_nonnull_average(NET, STA, newdf, stlat, stlon, SAVEpath, YearRange, ver)
        print(meanphiRC, stdphiRC, meandtRC, stddtRC, meanphiSC, stdphiSC, meandtSC, stddtSC, noNull, noNonNull)
        if np.isnan(meanphiRC) == False:
            allSTA['RCphi'].append(round(meanphiRC)); allSTA['RCdphi'].append(round(stdphiRC))
            allSTA['RCdt'].append(float(round(meandtRC,2))); allSTA['RCddt'].append( float(round(stddtRC,2)))
            allSTA['SCphi'].append(round(meanphiSC)); allSTA['SCdphi'].append(round(stdphiSC))
            allSTA['SCdt'].append(float(round(meandtSC,2))); allSTA['SCddt'].append( float(round(stddtSC,2)))
            allSTA['Null'].append(noNull); allSTA['nonNull'].append(noNonNull)
        else: 
            allSTA['RCphi'].append(meanphiRC); allSTA['RCdphi'].append(stdphiRC)
            allSTA['RCdt'].append(meandtRC); allSTA['RCddt'].append(stddtRC)
            allSTA['SCphi'].append(meanphiSC); allSTA['SCdphi'].append(stdphiSC)
            allSTA['SCdt'].append(meandtSC); allSTA['SCddt'].append(stddtSC)
            allSTA['Null'].append(noNull); allSTA['nonNull'].append(noNonNull)

#         plot_null_PCA09(NET, STA, newdf, stlat, stlon, SAVEpath, YearRange, meanphiSC, meandtSC, ver)
    #     plot_null_PCA_angle_small(NET, STA, newdf, stlat, stlon, SAVEpath, YearRange, ver)
    #     plot_null_PCA_angle_large(NET, STA, newdf, stlat, stlon, SAVEpath, YearRange, ver)  
    #     plot_null_angle(NET, STA, newdf, stlat, stlon, SAVEpath, YearRange, ver)  
    # stadf = pd.DataFrame(allSTA)
    # stadf.to_csv(f'{SAVEpath}/Station_SKS_{YearRange}average_v3.csv',index=False)


IU.GNI
140
GNI 20130514_003226 not used
GNI 20180828_223513 not used
GNI 20200613_210830 not used
GNI 20131123_074832 not used
51.5916510289 8.6297729849 0.680094796554 0.189481199595 52.9096756682 7.72794520897 0.75922327436 0.178579464105 111 24


In [25]:
stadf = pd.DataFrame(allSTA)
stadf.to_csv(f'{SAVEpath}/Station_SK(K)S_2010-2020average_v{ver}.csv',index=False)

In [27]:
SAVEpath

'/Volumes/home/Research/STEP/05_Station_result_statistics'

In [6]:
1326+187

1513